In [4]:
import dash
from dash import html, dcc, callback,get_asset_url
from dash.dependencies import Input, Output, State
import pandas as pd
import plotly.graph_objects as go
import numpy as np
from datetime import datetime, timedelta

####################################################################################
## Noise data ##
daily_noise = pd.read_csv("../Data/daily_noisedata_2022.csv")
daily_weather = pd.read_csv("../Data/daily_weatherdata_2022.csv")

####################################################################################
## path images ##

image_rainy = "cloud with rain.jpg"
image_sunny = "sunny-day.jpg"
image_cloudly = "Cloudly.jpg"

######################################################################################"
#gps data"
# Create dataframe with GPS coordinates
gps_data = {
    'description': ['MP 01: Naamsestraat 35  Maxim', 'MP 02: Naamsestraat 57 Xior', 'MP 03: Naamsestraat 62 Taste', 'MP 04: His & Hears', 'MP 05: Calvariekapel KU Leuven', 'MP 06: Parkstraat 2 La Filosovia', 'MP 07: Naamsestraat 81', 'MP08bis - Vrijthof'],
    'lat': [50.877121, 50.87650, 50.87590, 50.875237, 50.87452, 50.874078, 50.873808, 50.87873],
    'lon': [4.700708, 4.700632, 4.700262, 4.700071, 4.69985, 4.70005, 4.700007, 4.70115]
}
gps_df = pd.DataFrame(gps_data)

######################################################################################
# Merging noise data with GPS coordinates
merged_daily = pd.merge(daily_noise, gps_df, on='description', how='left')
#merged_monthly = pd.merge(monthly_noise, gps_df, on='description', how='left')

######################################################################################
#setup arrays for months
months_for_cum_months = [0,31,28,31,30,31,30,31,31,30,31,30,31]
cumulative_months = np.cumsum(months_for_cum_months).tolist()

######################################################################################
#cumulative day
merged_daily["day_cum"] = merged_daily.apply(lambda row: cumulative_months[row["month"]-1] + row["day"], axis=1)
daily_weather["day_cum"] = daily_weather.apply(lambda row: cumulative_months[np.int64(row["Month"])-1] + row["Day"], axis=1)

# select only certain columns of weather data
daily_weather = daily_weather[["LC_RAININ", "LC_TEMP_QCL3", "day_cum"]]

######################################################################################
# Group the data by 'day_cum' and apply the scaler separately for each group
max_laeq = merged_daily["laeq"].max()
min_laeq = merged_daily["laeq"].min()

start_size = 0.3 #minimum size
step_size = 0.7 #maximum size of 1 (0.3+0.7)

def divide_by_max(x):
    #max_value = x.max()#for each column separately: not good all the circles more or less same size
    return x / max_laeq

def myMapping(x):
    return start_size + step_size*(x-min_laeq)/(max_laeq-min_laeq)

merged_daily['laeq_std'] = merged_daily.groupby('day_cum')['laeq'].transform(myMapping)

######################################################################################
# Group the data by 'day_cum' and apply the scaler separately for each group
#same for lamax
max_lamax = merged_daily["lamax"].max()
min_lamax = merged_daily["lamax"].min()

start_size = 0.3 #minimum size
step_size = 0.7 #maximum size of 1 (0.3+0.7)

def myMapping(x):
    return start_size + step_size*(x-min_lamax)/(max_lamax-min_lamax)

merged_daily['lamax_std'] = merged_daily.groupby('day_cum')['lamax'].transform(myMapping)

######################################################################################
#Change the locations names 
replacements = {
    'MP 01: Naamsestraat 35  Maxim': 'Maxim (Naamsestraat 35)',
    'MP 02: Naamsestraat 57 Xior': 'Xior (Naamsestraat 57)',
    'MP 03: Naamsestraat 62 Taste': 'Taste (Naamsestraat 62)',
    'MP 04: His & Hears': 'His & Hears',
    'MP 05: Calvariekapel KU Leuven': 'Calvariekapel KU Leuven',
    'MP 06: Parkstraat 2 La Filosovia': 'La Filosovia (Parkstraat 2)',
    'MP 07: Naamsestraat 81': 'Naamsestraat 81',
    'MP08bis - Vrijthof': 'Vrijthof (stadhuis Leuven)'
}
# Replace the values in the "description" column
merged_daily['description'] = merged_daily['description'].replace(replacements)


######################################################################################
# Merge noise and weather data together in a final dataset
merged_data = pd.merge(merged_daily, daily_weather, on='day_cum', how='left')



PageError: `dash.register_page()` must be called after app instantiation